In [86]:
import os 
import pandas as pd

credentials_path="../credentials/my-project-428316-bad106950325.json"

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = os.path.join(os.getcwd(), credentials_path)

%reload_ext google.cloud.bigquery

# Viewing Data Sample

In [2]:
%%bigquery --use_rest_api 
SELECT *
FROM bigquery-public-data.bbc_news.fulltext
LIMIT 5;

Query is running:   0%|          |

Downloading:   0%|          |

,body,title,filename,category
0,The global web blog community is being called ...,Global blogger action day called,bbc/tech/016.txt,tech
1,"The ""digital divide"" between rich and poor nat...",Global digital divide 'narrowing',bbc/tech/033.txt,tech
2,The current slew of sports games offers unpara...,Sporting rivals go to extra time,bbc/tech/056.txt,tech
3,Writing a Microsoft Word document can be a dan...,Warning over Windows Word files,bbc/tech/086.txt,tech
4,"Aid workers trying to house, feed and clothe m...",Satellite mapping aids Darfur relief,bbc/tech/223.txt,tech


# Checking Rows Count

In [3]:
%%bigquery --use_rest_api 
SELECT COUNT(*) as cnt
FROM bigquery-public-data.bbc_news.fulltext;

Query is running:   0%|          |

Downloading:   0%|          |

,cnt
0,2225


# Check duplicates

In [44]:
%%bigquery --use_rest_api
WITH duplicates AS (
SELECT
    title,
    COUNT(*) AS duplicate_count
FROM
    `bigquery-public-data.bbc_news.fulltext`
GROUP BY
    title
)

SELECT * FROM duplicates
WHERE duplicate_count > 1
ORDER BY duplicate_count;


Query is running:   0%|          |

Downloading:   0%|          |

,title,duplicate_count
0,DVD copy protection strengthened,2
1,Intel unveils laser breakthrough,2
2,Looks and music to drive mobiles,2
3,Millions to miss out on the net,2
4,Virus poses as Christmas e-mail,2
...,...,...
124,Day-Lewis set for Berlin honour,2
125,Sir Paul rocks Super Bowl crowds,2
126,Pop band Busted to 'take a break',2
127,Downloads enter US singles chart,2


In [54]:
%%bigquery --use_rest_api df
SELECT *, CHAR_LENGTH(body) as body_cnt
FROM `bigquery-public-data.bbc_news.fulltext`
WHERE title IN ('DVD copy protection strengthened', 'Intel unveils laser breakthrough');


Query is running:   0%|          |

Downloading:   0%|          |

In [58]:
def print_body(df:pd.DataFrame):
    titles = df["title"].unique().tolist()
    print(titles)
    string = ""
    for title in titles:
        df2 = df.copy()
        df2 = df2[df2['title'] == title]
        row_num = df2.shape[0]
        string = string + f"\n\n\nTitle:{title}\n"
        for i in range(row_num):
            string = string + f"""
****************************************
{['First text', 'Second text'][i]}
    {(str(df2.iloc[i]["body"]))}
                """
    print(string)
        
print_body(df)

['DVD copy protection strengthened', 'Intel unveils laser breakthrough']



Title:DVD copy protection strengthened

****************************************
First text
    DVDs will be harder to copy thanks to new anti-piracy measures devised by copy protection firm Macrovision.

The pirated DVD market is enormous because current copy protection was hacked more than five years ago. Macrovision says its new RipGuard technology will thwart most, but not all, of the current DVD ripping (copying) programs used to pirate DVDs. "RipGuard is designed to... reduce DVD ripping and the resulting supply of illegal peer to peer," said the firm. Macrovision said the new technology will work in "nearly all" current DVD players when applied to the discs, but it did not specify how many machines could have a problem with RipGuard. Some BBC News website users have expressed concerns that the new technology will mean that DVDs will not work on PCs running the operating system Linux. The new technology w

## Checking for body duplicates

In [168]:
%%bigquery --use_rest_api

WITH RankedData AS (
    SELECT
        *,
        ROW_NUMBER() OVER(PARTITION BY body ORDER BY body) AS row_number
    FROM 
        `bigquery-public-data.bbc_news.fulltext`
)

SELECT *
FROM RankedData
WHERE row_number > 1;


Query is running:   0%|          |

Downloading:   0%|          |

,body,title,filename,category,row_number
0,British and Irish Lions coach Clive Woodward s...,Lions blow to World Cup winners,bbc/sport/387.txt,sport,2
1,The Aviator has been tipped by UK bookmakers a...,Bookmakers back Aviator for Oscar,bbc/entertainment/086.txt,entertainment,2
2,Former Conservative leader William Hague says ...,Hague 'given up' his PM ambition,bbc/politics/285.txt,politics,2
3,Michael Howard has denied his shadow cabinet w...,Howard denies split over ID cards,bbc/politics/300.txt,politics,2
4,Microsoft has warned PC users to update their ...,Microsoft releases bumper patches,bbc/tech/007.txt,tech,2
...,...,...,...,...,...
128,Firms that flout rules on how ringtones and ot...,Tough rules for ringtone sellers,bbc/tech/382.txt,tech,2
129,Two of Britain's big trade unions could merge ...,'Super union' merger plan touted,bbc/politics/291.txt,politics,2
130,IBM is spending $100m (£52m) over the next thr...,IBM puts cash behind Linux push,bbc/tech/012.txt,tech,2
131,The first convictions for piracy over peer-to-...,US peer-to-peer pirates convicted,bbc/tech/075.txt,tech,2


## Filtering Duplicates

In [163]:
%%bigquery --use_rest_api
SELECT *
FROM (
  SELECT
      *,
      ROW_NUMBER()
          OVER (PARTITION BY body)
          row_number
  FROM 
    `bigquery-public-data.bbc_news.fulltext`
)
WHERE row_number = 1

Query is running:   0%|          |

Downloading:   0%|          |

,body,title,filename,category,row_number
0,Literally putting a face on technology could b...,Putting a face to 'Big Brother',bbc/tech/143.txt,tech,1
1,Soon you could go hunting via the net.\n\nA Te...,Remote control rifle range debuts,bbc/tech/202.txt,tech,1
2,The European Parliament has thrown out a bill ...,EU software patent law faces axe,bbc/tech/330.txt,tech,1
3,UK mobile owners continue to break records wit...,Text message record smashed again,bbc/tech/372.txt,tech,1
4,A proposed European law on software patents wi...,'No re-draft' for EU patent law,bbc/tech/048.txt,tech,1
...,...,...,...,...,...
2087,The Incredibles movie has beaten Shrek 2 to th...,Incredibles win animation awards,bbc/entertainment/332.txt,entertainment,1
2088,Three of the five nominees in the live-action ...,Briton wins short film Oscar,bbc/entertainment/369.txt,entertainment,1
2089,Veteran comedian and broadcaster Cyril Fletche...,Veteran comic Cyril Fletcher dies,bbc/entertainment/190.txt,entertainment,1
2090,Film director Quentin Tarantino is to direct a...,Tarantino to direct CSI episode,bbc/entertainment/365.txt,entertainment,1


# Character Length

## Average CharLength of Articles per Category

In [61]:
%%bigquery --use_rest_api 
SELECT
    Category,
    AVG(CHAR_LENGTH(body)) as body_len
FROM bigquery-public-data.bbc_news.fulltext
GROUP BY Category
ORDER BY body_len DESC;

Query is running:   0%|          |

Downloading:   0%|          |

,Category,body_len
0,tech,2943.024938
1,politics,2650.592326
2,business,1952.762745
3,entertainment,1894.699482
4,sport,1865.023483


# CharLength Avg, Min, And Max

In [154]:
%%bigquery --use_rest_api df
SELECT
    MIN(CHAR_LENGTH(body)) as body_len_min,
    AVG(CHAR_LENGTH(body)) as body_len_avg,
    MAX(CHAR_LENGTH(body)) as body_len_max,
FROM bigquery-public-data.bbc_news.fulltext;

Query is running:   0%|          |

Downloading:   0%|          |

In [156]:
tdf = df.transpose()
tdf.columns = ['value']
tdf['metric'] = tdf.index
tdf = tdf.reset_index(drop=True)
tdf['value']=tdf.value.apply(lambda x: int(x))
tdf

,value,metric
0,470,body_len_min
1,2231,body_len_avg
2,25453,body_len_max


# Checking Categories

In [7]:
%%bigquery --use_rest_api 
SELECT 
    DISTINCT Category,
    COUNT(*) OVER (PARTITION BY CATEGORY) as cnt_category
FROM bigquery-public-data.bbc_news.fulltext;


Query is running:   0%|          |

Downloading:   0%|          |

,Category,cnt_category
0,sport,511
1,business,510
2,tech,401
3,politics,417
4,entertainment,386


# Checking for a Potention PrimaryKey
## Checking if filename can be used as PK

In [8]:
%%bigquery --use_rest_api

WITH counted_filenames AS (
    SELECT 
        filename,
        COUNT(*) OVER (PARTITION BY filename) as cnt_filename
    FROM 
        bigquery-public-data.bbc_news.fulltext
)

SELECT *
FROM counted_filenames
WHERE cnt_filename > 1;


Query is running:   0%|          |

Downloading: |          |

,filename,cnt_filename


# Insights:

- filename can be used as a primary key (PK).
- title has duplicates, but upon investigation, it appears that most of them have modifications.
- The maximum expected input character length per article is about 25,000 characters.
- The average expected input character length per article is about 2,500 characters.